# pyvespa API

> Simplification, embeddings and consistency

- toc: true 
- badges: false
- comments: true
- categories: [pyvespa, embeddings]

While experimenting with embeddings in pyvespa, I noticed that I could make significant improvements on the pyvespa API:

* Simplify the API for the most common and basic usage (one `default` schema, one `default` query profile with `root` query profile type) while making it easy to configure more complex usage (multiple schemas, for example).

* Create a unified approach to define and use embeddings in the applications.

* Expose similar functionality to Applications not defined by the `pyvespa` API.

## Simpler API

### Default Schema

Before simplification:

In [ ]:
from vespa.package import Document, Field, Schema, FieldSet, RankProfile, ApplicationPackage

document = Document(
    fields=[
        Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
        Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
        Field(name = "body", type = "string", indexing = ["index", "summary"], index = "enable-bm25")        
    ]
)
msmarco_schema = Schema(
    name = "msmarco", 
    document = document, 
    fieldsets = [FieldSet(name = "default", fields = ["title", "body"])],
    rank_profiles = [RankProfile(name = "default", first_phase = "nativeRank(title, body)")]
)
app_package = ApplicationPackage(name = "msmarco", schema=msmarco_schema)

After simplification:

In [ ]:
from vespa.package import Field, FieldSet, RankProfile, ApplicationPackage

app_package = ApplicationPackage(name = "msmarco")

app_package.schema.add_field(        
    Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
    Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
    Field(name = "body", type = "string", indexing = ["index", "summary"], index = "enable-bm25")        
)
app_package.schema.add_field_set(FieldSet(name = "default", fields = ["title", "body"]))
app_package.schema.add_rank_profile(RankProfile(name = "default", first_phase = "nativeRank(title, body)"))

When required we can use a similar pattern to deal with multiple schemas use cases where the `.schema` attribute is short for `.schemas("default")`

In [ ]:
app_package.schemas("my_other_schema").add_field(        
    Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
    Field(name = "user_name", type = "string", indexing = ["attribute", "summary"]),
)

### Default query profile with root query profile type

Example of what we want to accomplish:

* query profile type

In [ ]:
<query-profile-type id="root">
  <field name="ranking.features.query(tensor_bert)" type="tensor&lt;float&gt;(x[768])" />
</query-profile-type>

* query profile

In [ ]:
<query-profile id="default" type="root">
  <field name="maxHits">1000</field>
</query-profile>

**Verbose API**:

In [ ]:
query_profile_type = QueryProfileType(
    id="root", 
    fields = [
        QueryTypeField(
            name="ranking.features.query(tensor_bert)",
            type="tensor<float>(x[768])"
        )
    ]
)
query_profile = QueryProfile(
    id="default", 
    type="root", 
    fields=[QueryField(name="maxHits", value=1000)]
)

app_package.add_query_profile_type(query_profile_type)
app_package.add_query_profile(query_profile)

**Simplified API**:

In [ ]:
app_package.query_profile_type.add_field(        
    QueryTypeField(
        name="ranking.features.query(tensor_bert)",
        type="tensor<float>(x[768])"
    )
)
app_package.query_profile.add_field(
    QueryTypeField(
        name="ranking.features.query(tensor_bert)",
        type="tensor<float>(x[768])"
    )
)

When required we can use a similar pattern to deal with multiple profiles where `.query_profile` attribute is short for `.query_profiles("default")`. Similarly for `.query_profile_type`.